In [6]:
import scipy.io
import numpy as np
from scopt import scopt
import portfolio as pr

In [10]:
problem_name='syn_1000_1500_10_50'
data = scipy.io.loadmat('../../data/' + problem_name + '.mat')
R = data['W']
results = {}

## Data preprocessing and parameters initialization

In [5]:
N, n = R.shape
Mf = 2
nu = 3
I = np.eye(n)
Iv = np.ones(n)

#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-10

#parameters for FW
FW_params={
    'iter_FW':50000,
    'line_search_tol':1e-10,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'backtracking',#,'estimate', #estimate L
    'use_two_phase':False,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol':1e-5,
    'conj_grad_iter':1000,
}

## Auxilary functions

In [8]:
func_x = lambda x: pr.portfolio(R,x)
grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * x, (1 - beta) * dot_product + beta * dot_product_s)
hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
extra_func = lambda x: R @ x
linear_orcale = lambda grad: pr.linear_oracle_simplex(grad)
prox_func = lambda x, L: pr.proj_simplex(x)

## Run SCOPT

In [9]:
 x, alpha_hist, Q_hist, time_hist, grad_hist = scopt(func_x,
          grad_x,
          hess_mult_x,
          hess_mult_vec_x,
          Mf,
          nu,
          prox_func,
          x0,  
          sc_params,                                              
          eps=terminate_tol,                                              
          print_every=10)
    
results[example]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    #'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

Fista err = 9.987e-06; Subiter = 444; subproblem converged!

iter =    1, stepsize = 5.670e-01, rdiff = 4.783e-01 , f = 0.0212893

Fista err = 8.721e-06; Subiter =  16; subproblem converged!

Fista err = 1.583e-06; Subiter =   2; subproblem converged!

Fista err = 6.048e-07; Subiter =   2; subproblem converged!

Fista err = 7.147e-07; Subiter =   2; subproblem converged!

Convergence achieved!
iter =    7, stepsize = 0.000e+00, rdiff = 0.000e+00,value=-4.47509

17.01478362083435


NameError: name 'results' is not defined